In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [5]:
message = "what are the five borough in nyc"
response = openai.chat.completions.create(model="gpt-4o-mini", 
                                          messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

New York City is comprised of five boroughs:

1. **Manhattan** - Known for its iconic skyline, Times Square, Central Park, and being a major financial and cultural center.
2. **Brooklyn** - Famous for its diverse neighborhoods, cultural scene, and landmarks like the Brooklyn Bridge and Coney Island.
3. **Queens** - The largest borough by area and known for its ethnic diversity, Flushing Meadows-Corona Park, and like the USTA Billie Jean King National Tennis Center.
4. **The Bronx** - Known for being the birthplace of hip-hop, the Bronx Zoo, and the New York Yankees.
5. **Staten Island** - Often referred to as the "forgotten borough," accessible by ferry and known for its parks and views of Manhattan.

These boroughs each have their own unique character and contribute to the rich tapestry of New York City.


In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [31]:
ed = Website("https://anthropic.com")

In [32]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [33]:
messages = [
    {"role": "system", "content": "You are a very bad at math assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

2 + 2 is 22! Just kidding, it's actually 4.


In [34]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home \\ Anthropic\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncement\nStatement from Dario Amodei on the Paris AI Action Summit\nRead the statement\nPolicy\nAnthropic Economic Index\nPolicy\nOur Responsible Scaling Pol

In [27]:
def summarize(url):
    webpage= Website(url)
    response = openai.chat.completions.create(
        model ="gpt-4o-mini",
        messages = messages_for(webpage)
    )
    return display(Markdown(response.choices[0].message.content))

In [28]:
summarize("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI research and safety company based in San Francisco, focused on developing and promoting the ethical use of artificial intelligence. The website highlights their key product, Claude 3.5 Sonnet, as their most advanced AI model, which is available for engagement through various applications and APIs.

## Key Features:
- **Claude 3.5 Sonnet**: The latest version of their AI model, promoting intelligent interactions.
- **API Access**: Tools to build custom applications powered by Claude.
- **Enterprise Solutions**: Tailored AI products for business needs.
- **Research Focus**: Commitment to AI safety, with initiatives like the "Anthropic Economic Index" and "Constitutional AI".

## Recent Announcements:
- **Paris AI Action Summit**: A statement from co-founder Dario Amodei addressing the company's perspectives and commitments at the summit.

## Company Values:
Anthropic emphasizes its interdisciplinary approach to ensure reliable and beneficial AI systems, with an open call for careers in the field.

For more information about their work, research, and career opportunities, you can explore the relevant sections on their website.